# 0.0 IMPORTS

In [55]:
import pandas as pd
import inflection
import regex as re

## 0.1 Helper Functions

## 0.2 Loading Data

In [56]:
df = pd.read_csv('../repos/train.csv')
df.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)'],
      dtype='object')

# 1.0 DESCRICAO DOS DADOS

In [57]:
df1 = df.copy()

## 1.1. Rename Columns


In [58]:
cols_old = ['ID', 'Delivery_person_ID', 'Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
            'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
            'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density', 'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
            'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)']
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

#rename
df1.columns = cols_new

## 1.2. Data Dimensions


In [59]:
print('Número de linhas: {}'.format(df1.shape[0]))
print('Número de colunas: {}'.format(df1.shape[1]))

Número de linhas: 45593
Número de colunas: 20


## 1.3. Data Types


In [60]:
df1.dtypes


id                              object
delivery_person_id              object
delivery_person_age             object
delivery_person_ratings         object
restaurant_latitude            float64
restaurant_longitude           float64
delivery_location_latitude     float64
delivery_location_longitude    float64
order_date                      object
time_orderd                     object
time_order_picked               object
weatherconditions               object
road_traffic_density            object
vehicle_condition                int64
type_of_order                   object
type_of_vehicle                 object
multiple_deliveries             object
festival                        object
city                            object
time_taken(min)                 object
dtype: object

## 1.4. Check NA


## 1.5. Fillout NA


In [61]:
# Tirar linhas com Nan
linhas_selecionadas = (df1['delivery_person_age'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()

linhas_selecionadas = (df1['road_traffic_density'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()

linhas_selecionadas = (df1['time_taken(min)'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()

linhas_selecionadas = (df1['festival'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()

linhas_selecionadas = (df1['city'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()

## 1.6. Change Types


In [62]:
# Converter a coluna age para int
df1['delivery_person_age'] = df1['delivery_person_age'].astype('int64')

# converter a coluna ratings de texto para número decimal (float)
df1['delivery_person_ratings'] = df1['delivery_person_ratings'].astype(
    float)

# converter a order_date de texto para data
df1['order_date'] = pd.to_datetime(df1['order_date'], format='%d-%m-%Y')

# converter multiple_deliveries de texto para numero inteiro (int)
linhas_selecionadas = (df1['multiple_deliveries'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype('int64')

# Remover o texto de números
df1.loc[:, "id"] = df1.loc[:, "id"].str.strip()
df1.loc[:, "road_traffic_density"] = df1.loc[:, "road_traffic_density"].str.strip()
df1.loc[:, "type_of_order"] = df1.loc[:, "type_of_order"].str.strip()
df1.loc[:, "type_of_vehicle"] = df1.loc[:, "type_of_vehicle"].str.strip()
df1.loc[:, "city"] = df1.loc[:, "city"].str.strip()
df1.loc[:, "festival"] = df1.loc[:, "festival"].str.strip()

# Limpando a coluna time_taken
df1['time_taken(min)'] = df1['time_taken(min)'].apply(
    lambda x: x.split('(min)')[1])
df1['time_taken(min)'] = df1['time_taken(min)'].astype('int64')


In [63]:
df1.dtypes

id                                     object
delivery_person_id                     object
delivery_person_age                     int64
delivery_person_ratings               float64
restaurant_latitude                   float64
restaurant_longitude                  float64
delivery_location_latitude            float64
delivery_location_longitude           float64
order_date                     datetime64[ns]
time_orderd                            object
time_order_picked                      object
weatherconditions                      object
road_traffic_density                   object
vehicle_condition                       int64
type_of_order                          object
type_of_vehicle                        object
multiple_deliveries                     int64
festival                               object
city                                   object
time_taken(min)                         int64
dtype: object

## 1.7. Descriptive Statistical